## Seldon V2 Kubernetes Examples


In [1]:
import os
os.environ["NAMESPACE"] = "seldon-mesh"

In [2]:
MESH_IP=!kubectl get svc seldon-mesh -n ${NAMESPACE} -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP
MESH_IP

'172.19.255.1'

## Explain Model

In [3]:
!cat ./models/income.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income
spec:
  storageUri: "gs://seldon-models/scv2/examples/mlserver_1.3.0/income/classifier"
  requirements:
  - sklearn


In [4]:
!kubectl create -f ./models/income.yaml -n ${NAMESPACE}

model.mlops.seldon.io/income created


In [5]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ${NAMESPACE}

model.mlops.seldon.io/income condition met


In [6]:
!kubectl get model income -n ${NAMESPACE} -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2023-03-10T10:54:55Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2023-03-10T10:54:55Z",
      "status": "True",
      "type": "Ready"
    }
  ],
  "replicas": 1
}


In [7]:
!seldon model infer income --inference-host ${MESH_IP}:80 \
     '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}' 

{
	"model_name": "income_1",
	"model_version": "1",
	"id": "3fa3a933-37dd-4e42-b5fd-f5bbc56f8bf4",
	"parameters": {},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1,
				1
			],
			"datatype": "INT64",
			"data": [
				0
			]
		}
	]
}


In [8]:
!cat ./models/income-explainer.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income-explainer
spec:
  storageUri: "gs://seldon-models/scv2/examples/mlserver_1.3.0/income/explainer"
  explainer:
    type: anchor_tabular
    modelRef: income


In [9]:
!kubectl create -f ./models/income-explainer.yaml -n ${NAMESPACE}

model.mlops.seldon.io/income-explainer created


In [10]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ${NAMESPACE}

model.mlops.seldon.io/income condition met
model.mlops.seldon.io/income-explainer condition met


In [11]:
!kubectl get model income-explainer -n ${NAMESPACE} -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2023-03-10T10:55:07Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2023-03-10T10:55:07Z",
      "status": "True",
      "type": "Ready"
    }
  ],
  "replicas": 1
}


In [12]:
!seldon model infer income-explainer --inference-host ${MESH_IP}:80 \
     '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}' 

{
	"model_name": "income-explainer_1",
	"model_version": "1",
	"id": "55af3c9b-defd-4a52-8047-cc532461e8a4",
	"parameters": {},
	"outputs": [
		{
			"name": "explanation",
			"shape": [
				1,
				1
			],
			"datatype": "BYTES",
			"parameters": {
				"content_type": "str"
			},
			"data": [
				"{\"meta\": {\"name\": \"AnchorTabular\", \"type\": [\"blackbox\"], \"explanations\": [\"local\"], \"params\": {\"seed\": 1, \"disc_perc\": [25, 50, 75], \"threshold\": 0.95, \"delta\": 0.1, \"tau\": 0.15, \"batch_size\": 100, \"coverage_samples\": 10000, \"beam_size\": 1, \"stop_on_first\": false, \"max_anchor_size\": null, \"min_samples_start\": 100, \"n_covered_ex\": 10, \"binary_cache_size\": 10000, \"cache_margin\": 1000, \"verbose\": false, \"verbose_every\": 1, \"kwargs\": {}}, \"version\": \"0.9.0\"}, \"data\": {\"anchor\": [\"Marital Status = Never-Married\", \"Relationship = Own-child\", \"Capital Gain <= 0.00\"], \"precision\": 0.9974489795918368, \"coverage\": 0.06853582554517133, \

In [13]:
!kubectl delete -f ./models/income.yaml -n ${NAMESPACE}
!kubectl delete -f ./models/income-explainer.yaml -n ${NAMESPACE}

model.mlops.seldon.io "income" deleted
model.mlops.seldon.io "income-explainer" deleted


## Explain Pipeline

In [14]:
!cat ./models/income.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income
spec:
  storageUri: "gs://seldon-models/scv2/examples/mlserver_1.3.0/income/classifier"
  requirements:
  - sklearn


In [15]:
!kubectl create -f ./models/income.yaml -n ${NAMESPACE}

model.mlops.seldon.io/income created


In [16]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ${NAMESPACE}

model.mlops.seldon.io/income condition met


In [17]:
!kubectl get model income -n ${NAMESPACE} -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2023-03-10T10:55:15Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2023-03-10T10:55:15Z",
      "status": "True",
      "type": "Ready"
    }
  ],
  "replicas": 1
}


In [18]:
!seldon model infer income --inference-host ${MESH_IP}:80 \
     '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}' 

{
	"model_name": "income_1",
	"model_version": "1",
	"id": "a7d69896-1396-4c15-92bb-bd586cb3572a",
	"parameters": {},
	"outputs": [
		{
			"name": "predict",
			"shape": [
				1,
				1
			],
			"datatype": "INT64",
			"data": [
				0
			]
		}
	]
}


In [19]:
!cat ./pipelines/income-v1.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: income-prod
spec:
  steps:
    - name: income
  output:
    steps:
    - income



In [20]:
!kubectl create -f ./pipelines/income-v1.yaml -n ${NAMESPACE}

pipeline.mlops.seldon.io/income-prod created


In [21]:
!kubectl wait --for condition=ready --timeout=300s pipeline --all -n ${NAMESPACE}

pipeline.mlops.seldon.io/income-prod condition met


In [22]:
!seldon pipeline infer income-prod --inference-host ${MESH_IP}:80 \
     '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}' 

{
	"model_name": "",
	"outputs": [
		{
			"data": [
				0
			],
			"name": "predict",
			"shape": [
				1,
				1
			],
			"datatype": "INT64"
		}
	]
}


In [23]:
!cat ./models/income-explainer-pipeline.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: income-explainer
spec:
  storageUri: "gs://seldon-models/scv2/examples/mlserver_1.3.0/income/explainer"
  explainer:
    type: anchor_tabular
    pipelineRef: income-prod


In [24]:
!kubectl create -f ./models/income-explainer-pipeline.yaml -n ${NAMESPACE}

model.mlops.seldon.io/income-explainer created


In [25]:
!kubectl wait --for condition=ready --timeout=300s model --all -n ${NAMESPACE}

model.mlops.seldon.io/income condition met
model.mlops.seldon.io/income-explainer condition met


In [26]:
!kubectl get model income-explainer -n ${NAMESPACE} -o jsonpath='{.status}' | jq -M .

{
  "conditions": [
    {
      "lastTransitionTime": "2023-03-10T10:55:34Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2023-03-10T10:55:34Z",
      "status": "True",
      "type": "Ready"
    }
  ],
  "replicas": 1
}


In [27]:
!seldon model infer income-explainer --inference-host ${MESH_IP}:80 \
     '{"inputs": [{"name": "predict", "shape": [1, 12], "datatype": "FP32", "data": [[47,4,1,1,1,3,4,1,0,0,40,9]]}]}' 

{
	"model_name": "income-explainer_1",
	"model_version": "1",
	"id": "aff2498f-c3be-4056-b718-eaa2f714d75b",
	"parameters": {},
	"outputs": [
		{
			"name": "explanation",
			"shape": [
				1,
				1
			],
			"datatype": "BYTES",
			"parameters": {
				"content_type": "str"
			},
			"data": [
				"{\"meta\": {\"name\": \"AnchorTabular\", \"type\": [\"blackbox\"], \"explanations\": [\"local\"], \"params\": {\"seed\": 1, \"disc_perc\": [25, 50, 75], \"threshold\": 0.95, \"delta\": 0.1, \"tau\": 0.15, \"batch_size\": 100, \"coverage_samples\": 10000, \"beam_size\": 1, \"stop_on_first\": false, \"max_anchor_size\": null, \"min_samples_start\": 100, \"n_covered_ex\": 10, \"binary_cache_size\": 10000, \"cache_margin\": 1000, \"verbose\": false, \"verbose_every\": 1, \"kwargs\": {}}, \"version\": \"0.9.0\"}, \"data\": {\"anchor\": [\"Marital Status = Never-Married\", \"Relationship = Own-child\", \"Capital Gain <= 0.00\"], \"precision\": 0.9937629937629938, \"coverage\": 0.06853582554517133, \

In [28]:
!kubectl delete -f ./pipelines/income-v1.yaml -n ${NAMESPACE}
!kubectl delete -f ./models/income-explainer-pipeline.yaml -n ${NAMESPACE}
!kubectl delete -f ./models/income.yaml -n ${NAMESPACE}

pipeline.mlops.seldon.io "income-prod" deleted
model.mlops.seldon.io "income-explainer" deleted
model.mlops.seldon.io "income" deleted
